In [1]:
import keras
import sys
sys.path.append('C:/Users/320060820/experiments/')
import model_data
from model_gen import keras_cnn
import numpy as np
from tqdm import tnrange, tqdm_notebook

from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [2]:
def brute_force(model_obj, name, test_data, layer_names, bw_range, f_range, 
                         fix_quant=False, parameters=None, eval_metric='acc_drop'):
    if fix_quant and parameters is None:
        raise ValueError('parameters is empty')
    
    if not isinstance(layer_names, list):
        layer_names = [layer_names]
        
    for i in range(len(layer_names)):
        
        scores_matrix = []
        
        for j in tnrange(len(bw_range), desc=f'BW Layer {layer_names[i]}', leave=True):
            
            scores_row = []
            for k in tqdm_notebook(range(len(f_range)), desc='F', leave=False):
                
                quant_params = {
                    layer_names[i]: [bw_range[j], f_range[k]]
                }
                if fix_quant:
                    quant_params = {**quant_params, **parameters}
                
                keras_model = model_data.Model(name, test_data, model=model_obj.get_fxp_model(quant_params))
                scores_row.append(keras_model.evaluate_accuracy()[1])
                
            scores_matrix.append(scores_row)
            
        scores = np.array(scores_matrix)

        if i == 0:
            model_scores = scores
        else:
            model_scores = np.dstack((model_scores, scores))
        
        print(f'Layer {layer_names[i]} done.')
    
    model = model_data.Model(name, test_data, model=model_obj.get_float_model())
    s = model.evaluate_accuracy()[1]
    
    if eval_metric == 'acc_drop':
        model_scores = (s - model_scores) / s
        
    elif eval_metric == 'acc_ratio':
        model_scores = model_scores / s
        
    elif eval_metric == 'quant_acc':
        pass
    
    return model_scores

In [3]:
def analysis(input_shape, name, path_trained_weights, output_path, test_data, parameters):
    num_outputs = 10
    bw_range = np.arange(1, 11, 1)
    f_range = np.arange(-4, 10, 1)

    layer_names = [f'conv2d_{i}' for i in range(1, 5)] + ['dense_1']

    keras_model = keras_cnn.KerasCNN(input_shape, num_outputs, 
                                     path_trained_weights=path_trained_weights)

    scores_1 = brute_force(keras_model, name, test_data, 
                                    layer_names, bw_range, f_range)
    
    print('Step 1/5 complete')
    scores_2_1 = brute_force(keras_model, name, test_data, 
                                    layer_names[1:], bw_range, f_range,
                                    fix_quant=True, parameters=parameters['2_1'])
    print('Step 2.1/5 complete')
    scores_2_2 = brute_force(keras_model, name, test_data, 
                                    layer_names[1:], bw_range, f_range,
                                    fix_quant=True, parameters=parameters['2_2'])
    print('Step 2.2/5 complete')
    scores_2_3 = brute_force(keras_model, name, test_data, 
                                    layer_names[1:], bw_range, f_range,
                                    fix_quant=True, parameters=parameters['2_3'])
    print('Step 2.3/5 complete')
    scores_3 = brute_force(keras_model, name, test_data, 
                                    layer_names[2:], bw_range, f_range,
                                    fix_quant=True, parameters=parameters['3'])
    print('Step 3/5 complete')
    scores_4 = brute_force(keras_model, name, test_data, 
                                    layer_names[3:], bw_range, f_range,
                                    fix_quant=True, parameters=parameters['4'])
    print('Step 4/5 complete')
    scores_5 = brute_force(keras_model, name, test_data, 
                                    layer_names[4:], bw_range, f_range,
                                    fix_quant=True, parameters=parameters['5'])

    np.save(output_path + 'scores_1_keras', arr=scores_1)
    np.save(output_path + 'scores_2_1_keras', arr=scores_2_1)
    np.save(output_path + 'scores_2_2_keras', arr=scores_2_2)
    np.save(output_path + 'scores_2_3_keras', arr=scores_2_3)
    np.save(output_path + 'scores_3_keras', arr=scores_3)
    np.save(output_path + 'scores_4_keras', arr=scores_4)
    np.save(output_path + 'scores_5_keras', arr=scores_5)

In [4]:
base_path = 'C:/Users/320060820/experiments/test_models/'
output_path = 'C:/Users/320060820/experiments/outputs/Brute_Force_Analysis/Activations/'

# MNIST model
input_shape = (28, 28, 1)
path_trained_weights = base_path + 'mnist_custom/mnist_keras_w.h5'
name = 'mnist_keras'
from test_models.mnist_custom import data
_, test_data = data.get_data()
parameters = {
    '2_1': {
        'conv2d_1': [6, 7]
    },
    '2_2': {
        'conv2d_1': [5, 6]
    },
    '2_3': {
        'conv2d_1': [4, 5]
    },
    '3': {
        'conv2d_1': [6, 7],
        'conv2d_2': [5, 2]
    },
    '4': {
        'conv2d_1': [6, 7],
        'conv2d_2': [5, 2],
        'conv2d_3': [4, 0],
    },
    '5': {
        'conv2d_1': [6, 7],
        'conv2d_2': [5, 2],
        'conv2d_3': [4, 0],
        'conv2d_4': [3, 0]
    }
}
analysis(input_shape, name, path_trained_weights, output_path + 'Keras_MNIST/', test_data, parameters)
print('Analysis of MNIST model complete')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Layer conv2d_1 done.



Layer conv2d_2 done.



Layer conv2d_3 done.



Layer conv2d_4 done.


...


Layer conv2d_4 done.



Layer dense_1 done.
Step 3/5 complete



Layer conv2d_4 done.



Layer dense_1 done.
Step 4/5 complete



Layer dense_1 done.
Analysis of MNIST model complete


In [5]:
# CIFAR10 model
input_shape = (32, 32, 3)
path_trained_weights = base_path + 'cifar_keras/cifar_keras_w.h5'
name = 'cifar_keras'
from test_models.cifar_keras import data
_, test_data = data.get_data()
parameters = {
    '2_1': {
        'conv2d_1': [7, 8]
    },
    '2_2': {
        'conv2d_1': [6, 7]
    },
    '2_3': {
        'conv2d_1': [5, 6]
    },
    '3': {
        'conv2d_1': [7, 8],
        'conv2d_2': [4, 1]
    },
    '4': {
        'conv2d_1': [7, 8],
        'conv2d_2': [4, 1],
        'conv2d_3': [5, 1]
    },
    '5': {
        'conv2d_1': [7, 8],
        'conv2d_2': [4, 1],
        'conv2d_3': [5, 1],
        'conv2d_4': [5, 0]
    }
}

analysis(input_shape, name, path_trained_weights, output_path + 'Keras_CIFAR10/', test_data, parameters)
print('Analysis of CIFAR10 model complete')


Layer conv2d_1 done.



Layer conv2d_2 done.



Layer conv2d_3 done.



Layer conv2d_4 done.



Layer dense_1 done.
Step 1/5 complete


...


Layer conv2d_4 done.



Layer dense_1 done.
Step 3/5 complete



Layer conv2d_4 done.



Layer dense_1 done.
Step 4/5 complete



Layer dense_1 done.
Analysis of CIFAR10 model complete


In [6]:
# SVHN model
###### OLD PARAMETERS. NEW ONES IN THE SVHN NOTEBOOK
input_shape = (32, 32, 3)
path_trained_weights = base_path + 'svhn_keras/svhn_keras_w.h5'
name = 'svhn_keras'
from test_models.svhn_keras import data
_, test_data = data.get_data()
parameters = {
    '2_1': {
        'conv2d_1': [6, 7]
    },
    '2_2': {
        'conv2d_1': [5, 6]
    },
    '2_3': {
        'conv2d_1': [4, 5]
    },
    '3': {
        'conv2d_1': [7, 8],
        'conv2d_2': [4, 1]
    },
    '4': {
        'conv2d_1': [6, 7],
        'conv2d_2': [4, 1],
        'conv2d_3': [4, 0]
    },
    '5': {
        'conv2d_1': [6, 7],
        'conv2d_2': [4, 1],
        'conv2d_3': [4, 0],
        'conv2d_4': [5, 0]
    }
}

analysis(input_shape, name, path_trained_weights, output_path + 'Keras_SVHN/', test_data, parameters)
print('Analysis of SVHN model complete')


Layer conv2d_1 done.



Layer conv2d_2 done.



Layer conv2d_3 done.



Layer conv2d_4 done.



Layer dense_1 done.
Step 1/5 complete


...


Layer conv2d_4 done.



Layer dense_1 done.
Step 3/5 complete



Layer conv2d_4 done.



Layer dense_1 done.
Step 4/5 complete



Layer dense_1 done.
Analysis of SVHN model complete


In [7]:
# Fashion-MNIST model
input_shape = (28, 28, 1)
path_trained_weights = base_path + 'fashion_keras/fashion_keras_w.h5'
name = 'fashion_keras'
from test_models.fashion_keras import data
_, test_data = data.get_data()
parameters = {
    '2_1': {
        'conv2d_1': [7, 8]
    },
    '2_2': {
        'conv2d_1': [6, 7]
    },
    '2_3': {
        'conv2d_1': [4, 5]
    },
    '3': {
        'conv2d_1': [7, 8],
        'conv2d_2': [4, 2]
    },
    '4': {
        'conv2d_1': [7, 8],
        'conv2d_2': [4, 2],
        'conv2d_3': [4, 0]
    },
    '5': {
        'conv2d_1': [7, 8],
        'conv2d_2': [4, 2],
        'conv2d_3': [4, 0],
        'conv2d_4': [4, 1]
    }
}

analysis(input_shape, name, path_trained_weights, output_path + 'Keras_Fashion/', test_data, parameters)
print('Analysis of Fashion model complete')


Layer conv2d_1 done.



Layer conv2d_2 done.



Layer conv2d_3 done.



Layer conv2d_4 done.



Layer dense_1 done.
Step 1/5 complete


...


Layer conv2d_4 done.



Layer dense_1 done.
Step 3/5 complete



Layer conv2d_4 done.



Layer dense_1 done.
Step 4/5 complete



Layer dense_1 done.
Analysis of Fashion model complete
